In [3]:
# --- Install dependencies ---
!pip install gradio opencv-python-headless matplotlib tqdm pytransform3d


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/60.2 MB ? eta -:--:--
   ------ --------------------------------- 9.7/60.2 MB 47.7 MB/s eta 0:00:02
   ---------- ----------------------------- 16.5/60.2 MB 40.4 MB/s eta 0:00:02
   --------------- ------------------------ 23.3/60.2 MB 37.3 MB/s eta 0:00:01
   -------------------- ------------------- 30.4/60.2 MB 36.4 MB/s eta 0:00:01
   ------------------------- -------------- 38.0/60.2 MB 36.3 MB/s eta 0:00:01
   ------------------------------ --------- 45.6/60.2 MB 36.6 MB/s eta 0:00:01
   ------------------------------------ --- 54.3/60.2 MB 37.2 MB/s eta 0:00:01
   ---------------------------------------  60.0/60.2 MB 37.6 MB/s eta 0:00:01
   ---------------------------------------- 60.2/60.2 MB 34.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
   ------ --------------------------------- 6.6/38.9 MB 32.3 MB/s eta 0:


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\blake\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
# Camera_Calibration_UI.ipynb

import gradio as gr
import os
import numpy as np
import cv2
from calibration.chessboard import find_corners_in_folder
from calibration.core import calibrate, undistort_image
from calibration.calib_io import save_calibration_json, load_calibration_json
from calibration.overlay import overlay_axes_on_calibration_images
from calibration.pose_viz import visualize_calibration_poses

# -------------------------------------------------------------------
# Helper functions for Gradio
# -------------------------------------------------------------------

def save_uploaded_images(files, save_dir="data/images"):
    """Save uploaded .jpeg/.jpg files to the project images folder."""
    os.makedirs(save_dir, exist_ok=True)
    saved = []
    for f in files:
        fname = os.path.basename(f)
        path = os.path.join(save_dir, fname)
        os.replace(f, path)  # move temp file into project folder
        saved.append(path)
    return f"✅ Saved {len(saved)} images to {save_dir}", save_dir

def run_calibration(images_dir, cols, rows, square_size):
    pattern_size = (cols, rows)
    try:
        imgpoints, image_size, used_paths = find_corners_in_folder(images_dir, pattern_size)
    except Exception as e:
        return f"Error: {str(e)}", None, None
    
    calib = calibrate(pattern_size, square_size, imgpoints, image_size)
    out_json = os.path.join(images_dir, "calibration.json")
    save_calibration_json(calib, out_json)
    
    summary = (
        f"RMS reprojection error: {calib['rms']:.4f}\n"
        f"Image size: {calib['image_size']}\n"
        f"Intrinsic matrix K:\n{np.array(calib['K'])}\n\n"
        f"Distortion coefficients:\n{np.array(calib['dist'])}\n"
    )
    return summary, out_json, used_paths[0] if used_paths else None


def preview_undistortion(calibration_json, image_path):
    if calibration_json is None or not os.path.exists(calibration_json):
        return "Calibration JSON not found", None
    
    calib = load_calibration_json(calibration_json)
    K, dist = calib["_K_np"], calib["_dist_np"]
    img = cv2.imread(image_path)
    und, _ = undistort_image(img, K, dist)
    side_by_side = np.hstack([img, und])
    preview_path = os.path.join(os.path.dirname(calibration_json), "undistort_preview.jpg")
    cv2.imwrite(preview_path, side_by_side)
    return "Preview generated", preview_path


def generate_axes_overlays(calibration_json, images_dir, output_dir="axes_results", max_images=8):
    try:
        paths = overlay_axes_on_calibration_images(calibration_json, images_dir, output_dir, max_images=max_images)
        return f"Created {len(paths)} overlay images", paths  # return all, not just 3
    except Exception as e:
        return f"Error generating overlays: {e}", None


def generate_pose_visualization(calibration_json, output_dir="pose_results"):
    try:
        figs = visualize_calibration_poses(calibration_json, output_dir, show_3d=True, show_2d=True)
        out_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".png")]
        return f"Generated {len(out_files)} visualization plots", out_files
    except Exception as e:
        return f"Error generating poses: {e}", None

# -------------------------------------------------------------------
# Build Gradio Interface
# -------------------------------------------------------------------

with gr.Blocks() as demo:
    gr.Markdown("# 📷 Camera Calibration with Gradio UI")
    
    with gr.Tab("1. Upload Images"):
        uploads = gr.File(
            label="Upload Chessboard Images (.jpeg or .jpg)",
            file_types=[".jpeg", ".jpg"],  # accept both
            file_count="multiple",         # allow multiple uploads
            type="filepath"                # return file paths
        )
        upload_btn = gr.Button("Save Uploaded Images")
        upload_status = gr.Textbox(label="Upload Status")
        images_dir = gr.Textbox(label="Images Folder", value="data/images", interactive=False)
        
        upload_btn.click(save_uploaded_images, inputs=[uploads], outputs=[upload_status, images_dir])
    
    with gr.Tab("2. Calibration"):
        cols = gr.Number(label="Columns (inner corners)", value=9)
        rows = gr.Number(label="Rows (inner corners)", value=6)
        square_size = gr.Number(label="Square size (mm)", value=25.0)
        calib_btn = gr.Button("Run Calibration")
        calib_output = gr.Textbox(label="Calibration Results")
        calib_json = gr.File(label="Calibration JSON", interactive=False)
        sample_img = gr.Image(label="Sample Image Used", type="filepath")
        
        calib_btn.click(
            run_calibration, 
            inputs=[images_dir, cols, rows, square_size],
            outputs=[calib_output, calib_json, sample_img]
        )
    
    with gr.Tab("3. Undistortion Preview"):
        img_in = gr.Image(label="Choose Image", type="filepath")
        undist_btn = gr.Button("Preview Undistortion")
        undist_status = gr.Textbox(label="Status")
        undist_img = gr.Image(label="Preview")
        
        undist_btn.click(
            preview_undistortion,
            inputs=[calib_json, img_in],
            outputs=[undist_status, undist_img]
        )
    
    with gr.Tab("4. Axes Overlays"):
        overlay_btn = gr.Button("Generate Axes Overlays (8 images)")
        overlay_status = gr.Textbox(label="Status")
        overlay_imgs = gr.Gallery(label="Overlay Samples", columns=4, height="auto")

        overlay_btn.click(
            generate_axes_overlays,
            inputs=[calib_json, images_dir],
            outputs=[overlay_status, overlay_imgs]
        )

    with gr.Tab("5. Camera Pose Visualization"):
        pose_btn = gr.Button("Generate 3D/2D Pose Visualizations")
        pose_status = gr.Textbox(label="Status")
        pose_imgs = gr.Gallery(label="Pose Plots", columns=2, height="auto")

        pose_btn.click(
            generate_pose_visualization,
            inputs=[calib_json],
            outputs=[pose_status, pose_imgs]
        )

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7868


OSError: [WinError 225] Operation did not complete successfully because the file contains a virus or potentially unwanted software: 'C:\\Users\\blake\\.cache\\huggingface\\gradio\\frpc\\frpc_windows_amd64_v0.3'